In [1]:
import pandas as pd 
import numpy as np  
import re
import random
import json 

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

from selenium.webdriver.common.action_chains import ActionChains
from time import sleep

from bs4 import BeautifulSoup
import time
from itertools import product

path = 'C:/Users/GMEU0170/OneDrive - Hyundai Motor Europe/GME_CD/Documents/2.__Code/_.py/web_scraping/selenium/'

In [2]:
# Set up Chrome options (headless mode is optional)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

# Use webdriver-manager to install and manage ChromeDriver
service = Service(ChromeDriverManager().install())

In [3]:
# User-Agent list for rotation
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15",
    # Add more user agents as needed
]

def get_driver(service):
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--blink-settings=imagesEnabled=false")
    ua = random.choice(user_agents)
    chrome_options.add_argument(f"user-agent={ua}")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)
    # Explicitly set the page load strategy to "normal"
    chrome_options.page_load_strategy = "normal"
    
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    driver.implicitly_wait(10)
    return driver

# driver = get_driver(service)

In [42]:
driver = get_driver(service)

url = "https://configure.bmw.de/de_DE/configure/G60/41HG/FKTSW,P0C4P,S0230,S02PA,S02VB,S02VD,S02VV,S0302,S033B,S03DK,S03DP,S03GT,S03M2,S03MF,S0420,S0428,S043R,S0459,S0488,S0494,S04GQ,S04NB,S04T2,S04U6,S04U9,S04UR,S04V1,S0548,S0552,S05AS,S05AV,S05DM,S0654,S0674,S06AE,S06AF,S06AK,S06C4,S06NX,S06PA,S0710,S0754,S0760,S0775,S07M9,S0801,S0851,S0879,S08R3,S08R9,S08TF,S08WN,S09QV,S09T1,S09T2,S09TB/SE000001"

driver.get(url)

In [43]:
js_find_and_click_cookie = """
    function findAndClickAccept(node) {
        if (!node) return false;

        // Check if it's a button or span containing the "Alle akzeptieren" text
        if ((node.tagName === 'BUTTON' || node.tagName === 'SPAN') && node.innerText.includes("Alle akzeptieren")) {
            node.click();
            return true;
        }

        // Traverse shadow DOM
        if (node.shadowRoot) {
            const found = findAndClickAccept(node.shadowRoot);
            if (found) return true;
        }

        // Traverse regular child elements
        for (const child of node.children || []) {
            const found = findAndClickAccept(child);
            if (found) return true;
        }

        return false;
    }
    return findAndClickAccept(document.body);
"""

# Execute the JavaScript in Selenium
clicked = driver.execute_script(js_find_and_click_cookie)

if clicked:
    print("✅ 'Alle akzeptieren' button clicked.")
else:
    print("❌ Could not find 'Alle akzeptieren' button.")


✅ 'Alle akzeptieren' button clicked.


In [44]:
js_find_and_click = """
function findAndClickButton(node) {
    if (!node) return null;

    if (node.tagName === "BUTTON" && node.innerText.includes("Finanzieren & Leasen")) {
        node.click();
        return true;
    }

    if (node.shadowRoot) {
        let found = findAndClickButton(node.shadowRoot);
        if (found) return true;
    }

    for (const child of node.children || []) {
        let found = findAndClickButton(child);
        if (found) return true;
    }

    return false;
}
return findAndClickButton(document.body);
"""

clicked = driver.execute_script(js_find_and_click)
if clicked:
    print("✅ Button clicked successfully!")
else:
    print("❌ Button not found!")

✅ Button clicked successfully!


In [33]:
# JavaScript to recursively find the <select> element
js_find_select = """
    function findSelect(node) {
        if (!node) return null;
        if (node.tagName === "SELECT" && node.classList.contains("sf-cf-grouped-params")) {
            return node;
        }
        if (node.shadowRoot) {
            const result = findSelect(node.shadowRoot);
            if (result) return result;
        }
        for (const child of node.children || []) {
            const result = findSelect(child);
            if (result) return result;
        }
        return null;
    }
    return findSelect(document.body);
"""

# Get the <select> element from shadow DOM
select_element = driver.execute_script(js_find_select)

In [34]:
html = driver.execute_script("return arguments[0].outerHTML;", select_element)
print(html)

<select id="selectid-17470598033376" class="select-class sf-cf-grouped-params" aria-label="Laufzeit" data-id=""><option value="24">24 Monate</option><option value="27">27 Monate</option><option value="30">30 Monate</option><option value="33">33 Monate</option><option value="36" selected="">36 Monate</option><option value="42">42 Monate</option><option value="48">48 Monate</option></select>


In [45]:
### Full options
# JavaScript to find all <select> elements recursively in shadow DOMs
js_find_all_selects = """
function findAllSelects(node) {
    let results = [];
    if (!node) return results;

    if (node.tagName === "SELECT") {
        results.push(node);
    }

    if (node.shadowRoot) {
        results = results.concat(findAllSelects(node.shadowRoot));
    }

    for (const child of node.children || []) {
        results = results.concat(findAllSelects(child));
    }

    return results;
}
return findAllSelects(document.body);
"""

# Execute JS to get all <select> elements
select_elements = driver.execute_script(js_find_all_selects)

print(f"🔍 Found {len(select_elements)} <select> elements.\n")

# Loop through all found <select> elements and print their options
for i, select in enumerate(select_elements):
    label = driver.execute_script("return arguments[0].getAttribute('aria-label') || arguments[0].id || 'Unknown';", select)
    options = driver.execute_script("""
        return Array.from(arguments[0].options).map(o => ({
            value: o.value,
            text: o.textContent.trim()
        }));
    """, select)

    print(f"🟦 Select {i+1}: {label}")
    for option in options:
        print(f"  • {option['text']} ({option['value']})")
    print()


🔍 Found 4 <select> elements.

🟦 Select 1: Weitere Finanzierungs- und Leasingbeispiele:
  • Das Ziel-Finanzierungsbeispiel der BMW Bank GmbH (1008542)
  • Das Leasingbeispiel der BMW Bank GmbH (1008954)

🟦 Select 2: Laufzeit
  • 24 Monate (24)
  • 27 Monate (27)
  • 30 Monate (30)
  • 33 Monate (33)
  • 36 Monate (36)
  • 42 Monate (42)
  • 48 Monate (48)

🟦 Select 3: Laufleistung p.a.
  • 5.000 km (5000)
  • 10.000 km (10000)
  • 15.000 km (15000)
  • 20.000 km (20000)
  • 25.000 km (25000)
  • 30.000 km (30000)
  • 35.000 km (35000)
  • 40.000 km (40000)
  • 45.000 km (45000)
  • 50.000 km (50000)
  • 55.000 km (55000)
  • 60.000 km (60000)
  • 65.000 km (65000)
  • 70.000 km (70000)
  • 75.000 km (75000)
  • 80.000 km (80000)

🟦 Select 4: Auswahl Leistungsumfang:
  • Leasingratenversicherung Tod und Arbeitsunfähigkeit * (Leasingratenversicherung Tod und Arbeitsunfähigkeit *)
  • Leasingratenversicherung Tod und Arbeitsunfähigkeit, Arbeitslosigkeit/Schwere Krankheit ** (Leasingratenve

In [ ]:
# JavaScript to find <select> elements with specific aria-labels
js_find_target_selects = """
    function findMatchingSelects(node, labels = [], result = []) {
        if (!node) return result;
        if (node.tagName === "SELECT") {
            const label = node.getAttribute("aria-label");
            if (label === "Laufzeit" || label === "Laufleistung p.a.") {
                result.push(node);
            }
        }
        if (node.shadowRoot) findMatchingSelects(node.shadowRoot, labels, result);
        for (const child of node.children || []) {
            findMatchingSelects(child, labels, result);
        }
        return result;
    }
    return findMatchingSelects(document.body);
"""
# Get the matching <select> elements (Laufzeit and Laufleistung p.a.)
select_elements = driver.execute_script(js_find_target_selects)

# Get options for each select
all_options = []
for sel in select_elements:
    values = driver.execute_script("return Array.from(arguments[0].options).map(o => o.value);", sel)
    all_options.append(values)
    
all_options

[['24', '27', '30', '33', '36', '42', '48'],
 ['5000',
  '10000',
  '15000',
  '20000',
  '25000',
  '30000',
  '35000',
  '40000',
  '45000',
  '50000',
  '55000',
  '60000',
  '65000',
  '70000',
  '75000',
  '80000']]

In [50]:
from itertools import product

# Loop through all combinations
for combo in product(*all_options):
    print(f"\n🔁 Trying combo: {combo}")

    # Re-fetch the two target selects again to avoid stale elements
    select_elements = driver.execute_script(js_find_target_selects)

    for i, sel in enumerate(select_elements):
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", sel)
        sel.click()
        sleep(0.5)

        # Find index of desired value
        index = driver.execute_script("""
            const options = Array.from(arguments[0].options);
            return options.findIndex(opt => opt.value === arguments[1]);
        """, sel, combo[i])

        if index == -1:
            continue

        # Use ARROW keys to move to correct option
        actions = ActionChains(driver)
        actions.send_keys(Keys.HOME)
        for _ in range(index):
            actions.send_keys(Keys.DOWN)
        actions.send_keys(Keys.RETURN).perform()
        sleep(2)

    # Extract result values
    js_extract_label_and_value = """
    function findNumbersWithLabels(node) {
        let results = [];
        if (node.tagName === "SF-CF-NUMBER") {
            let span = node.querySelector("span.sf-cf-number");
            if (span) {
                results.push({
                    parameterId: node.getAttribute("data-parameterid"),
                    value: span.textContent.trim()
                });
            }
        }
        if (node.shadowRoot) {
            results = results.concat(findNumbersWithLabels(node.shadowRoot));
        }
        for (const child of node.children || []) {
            results = results.concat(findNumbersWithLabels(child));
        }
        return results;
    }
    return findNumbersWithLabels(document.body);
    """
    results = driver.execute_script(js_extract_label_and_value)

    desired_ids = {
        "Price_Leasing", "Term", "DepositAmountLeasing",
        "MonthlyPaymentWithoutServices", "Mileage", "TotalPaymentWithoutServices"
    }

    for item in results:
        if item["parameterId"] in desired_ids:
            print(f"  • {item['parameterId']}: {item['value']}")


🔁 Trying combo: ('24', '5000')
  • Price_Leasing: 101.770,00 €
  • Term: 24 Monate
  • DepositAmountLeasing: 25.200,17 €
  • MonthlyPaymentWithoutServices: 859,64 €
  • Mileage: 5.000 km
  • TotalPaymentWithoutServices: 45.831,53 €

🔁 Trying combo: ('24', '10000')
  • Price_Leasing: 101.770,00 €
  • Term: 24 Monate
  • DepositAmountLeasing: 25.200,17 €
  • MonthlyPaymentWithoutServices: 939,99 €
  • Mileage: 10.000 km
  • TotalPaymentWithoutServices: 47.759,93 €

🔁 Trying combo: ('24', '15000')
  • Price_Leasing: 101.770,00 €
  • Term: 24 Monate
  • DepositAmountLeasing: 25.200,17 €
  • MonthlyPaymentWithoutServices: 980,16 €
  • Mileage: 15.000 km
  • TotalPaymentWithoutServices: 48.724,01 €

🔁 Trying combo: ('24', '20000')
  • Price_Leasing: 101.770,00 €
  • Term: 24 Monate
  • DepositAmountLeasing: 25.200,17 €
  • MonthlyPaymentWithoutServices: 1.020,33 €
  • Mileage: 20.000 km
  • TotalPaymentWithoutServices: 49.688,09 €

🔁 Trying combo: ('24', '25000')
  • Price_Leasing: 101.770,

KeyboardInterrupt: 

In [24]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

# Click the <select> element
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", select_element)
select_element.click()
sleep(1)

# Send DOWN + ENTER keys
actions = ActionChains(driver)
actions.send_keys(Keys.ARROW_UP).send_keys(Keys.RETURN).perform()


In [ ]:
# Click the <select> element
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", select_element)
select_element.click()
sleep(1)

# Send DOWN + ENTER keys
actions = ActionChains(driver)
actions.send_keys(Keys.ARROW_UP).send_keys(Keys.RETURN).perform()

# Get number of options in the dropdown
option_count = driver.execute_script("return arguments[0].options.length;", select_element)

for i in range(option_count):
    print(f"🔁 Selecting option {i + 1}/{option_count}")

    # Scroll into view and click the dropdown
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", select_element)
    select_element.click()
    sleep(1)

    # Navigate to the desired option with ARROW keys and select it
    actions = ActionChains(driver)
    actions.send_keys(Keys.HOME)  # Move to the first option
    for _ in range(i):
        actions.send_keys(Keys.DOWN)
    actions.send_keys(Keys.RETURN)
    actions.perform()

    sleep(3)  # Wait for UI to update

    # Extract updated price and other sf-cf-number values
    js_extract_label_and_value = """
    function findNumbersWithLabels(node) {
        let results = [];
        if (node.tagName === "SF-CF-NUMBER") {
            let span = node.querySelector("span.sf-cf-number");
            if (span) {
                results.push({
                    parameterId: node.getAttribute("data-parameterid"),
                    value: span.textContent.trim()
                });
            }
        }
        if (node.shadowRoot) {
            results = results.concat(findNumbersWithLabels(node.shadowRoot));
        }
        for (const child of node.children || []) {
            results = results.concat(findNumbersWithLabels(child));
        }
        return results;
    }
    return findNumbersWithLabels(document.body);
    """

    results = driver.execute_script(js_extract_label_and_value)
    for item in results:
        print(f"  • Parameter ID: {item['parameterId']}, Value: {item['value']}")

🔁 Selecting option 1/7
  • Parameter ID: None, Value: 0,00 €
  • Parameter ID: None, Value: 461,48 €
  • Parameter ID: None, Value: 0 Monate
  • Parameter ID: None, Value: 24 Monate
  • Parameter ID: None, Value: 27 Monate
  • Parameter ID: None, Value: 30 Monate
  • Parameter ID: None, Value: 33 Monate
  • Parameter ID: None, Value: 36 Monate
  • Parameter ID: None, Value: 42 Monate
  • Parameter ID: None, Value: 48 Monate
  • Parameter ID: None, Value: 0 km
  • Parameter ID: None, Value: 5.000 km
  • Parameter ID: None, Value: 10.000 km
  • Parameter ID: None, Value: 15.000 km
  • Parameter ID: None, Value: 20.000 km
  • Parameter ID: None, Value: 25.000 km
  • Parameter ID: None, Value: 30.000 km
  • Parameter ID: None, Value: 35.000 km
  • Parameter ID: None, Value: 40.000 km
  • Parameter ID: None, Value: 45.000 km
  • Parameter ID: None, Value: 50.000 km
  • Parameter ID: None, Value: 55.000 km
  • Parameter ID: None, Value: 60.000 km
  • Parameter ID: None, Value: 65.000 km
  • 

### Testing text

In [34]:
driver = get_driver(service)
driver.get(url)

In [35]:
js_find_and_click_cookie = """
    function findAndClickAccept(node) {
        if (!node) return false;

        // Check if it's a button or span containing the "Alle akzeptieren" text
        if ((node.tagName === 'BUTTON' || node.tagName === 'SPAN') && node.innerText.includes("Alle akzeptieren")) {
            node.click();
            return true;
        }

        // Traverse shadow DOM
        if (node.shadowRoot) {
            const found = findAndClickAccept(node.shadowRoot);
            if (found) return true;
        }

        // Traverse regular child elements
        for (const child of node.children || []) {
            const found = findAndClickAccept(child);
            if (found) return true;
        }

        return false;
    }
    return findAndClickAccept(document.body);
"""

# Execute the JavaScript in Selenium
clicked = driver.execute_script(js_find_and_click_cookie)

if clicked:
    print("✅ 'Alle akzeptieren' button clicked.")
else:
    print("❌ Could not find 'Alle akzeptieren' button.")


✅ 'Alle akzeptieren' button clicked.


In [37]:
body = driver.find_element(By.TAG_NAME, "body")
all_text = body.text

print(all_text)

Verfügbare Neuwagen
Modell ändern
0 g/km
BMW iX2 eDrive20
0 g/km
Motoren
Modellvarianten
Exterieur
Interieur
Pakete & Sonderausstattungen
Laden
Services
Winterräder
Nächste Schritte
Motoren
BMW iX2 eDrive20
Vollelektrisch
Ab 50.300 €
Getriebe
Automatik Getriebe
Technische Daten
Alle Motorisierungen
Serienausstattung
Dieser BMW überzeugt mit einer großzügigen Serienausstattung im Exterieur und Interieur.
Serienausstattung entdecken
Modellvarianten
Wählen Sie Ihre Modellvariante
Serie
Ab 50.300 €
Mehr Details
Alle Modellvarianten
Exterieur
Außenfarben
Für Ihre Modellvariante
METALLIC
890 €
890 €
890 €
890 €
890 €
890 €
1.780 €
2.670 €
2.670 €
UNI
0 €
320 €
KEINE AUSWAHL VERFÜGBAR.
Diese Option erfordert weitere Anpassungen
Für andere Modellvarianten
BMW Individual Lackierungen entdecken
Räder
Informationen zur EU Reifenkennzeichnung finden Sie im Bereich „technische Daten“ auf der fahrzeugspezifischen Informationsseite.
Für Ihre Modellvariante
0 €
850 €
950 €
2.060 €
3.060 €
KEINE AUSWAH

NameError: name 'all_text' is not defined

# Full flow

In [17]:
driver = get_driver(service)

# url = "https://configure.bmw.de/de_DE/configure/U10/51GM/FBLAT,P0C68,S01EE,S0230,S02PA,S02VB,S02VD,S0302,S0428,S0478,S0494,S04GN,S04T2,S04U9,S04V1,S0548,S05AV,S05DM,S0654,S06AE,S06AF,S06PA,S0760,S0801,S0851,S0879,S08R3,S08R9,S08TF/SE000001"
url = "https://configure.bmw.de/de_DE/configure/G60/61HG/FKUSW,P0475,S0230,S02PA,S02VB,S02VD,S0302,S03DK,S03G3,S0428,S043G,S0488,S0494,S04T2,S04U9,S04UR,S04V1,S0548,S0552,S05AS,S05AV,S05DM,S0654,S0674,S06AE,S06AF,S06AK,S06C4,S06NX,S06PA,S0704,S0801,S0851,S0879,S08R3,S08R9,S08TF,S08WN,S09QV/SE000001"

driver.get(url)

sleep(2)

js_find_and_click_cookie = """
    function findAndClickAccept(node) {
        if (!node) return false;

        // Check if it's a button or span containing the "Alle akzeptieren" text
        if ((node.tagName === 'BUTTON' || node.tagName === 'SPAN') && node.innerText.includes("Alle akzeptieren")) {
            node.click();
            return true;
        }

        // Traverse shadow DOM
        if (node.shadowRoot) {
            const found = findAndClickAccept(node.shadowRoot);
            if (found) return true;
        }

        // Traverse regular child elements
        for (const child of node.children || []) {
            const found = findAndClickAccept(child);
            if (found) return true;
        }

        return false;
    }
    return findAndClickAccept(document.body);
"""

# Execute the JavaScript in Selenium
clicked = driver.execute_script(js_find_and_click_cookie)

if clicked:
    print("✅ 'Alle akzeptieren' button clicked.")
else:
    print("❌ Could not find 'Alle akzeptieren' button.")

sleep(2)

body = driver.find_element(By.TAG_NAME, "body")
all_text = body.text

# sleep(2)

js_find_and_click = """
    function findAndClickButton(node) {
        if (!node) return null;

        if (node.tagName === "BUTTON" && node.innerText.includes("Finanzieren & Leasen")) {
            node.click();
            return true;
        }

        if (node.shadowRoot) {
            let found = findAndClickButton(node.shadowRoot);
            if (found) return true;
        }

        for (const child of node.children || []) {
            let found = findAndClickButton(child);
            if (found) return true;
        }

        return false;
    }
    return findAndClickButton(document.body);
    """

clicked = driver.execute_script(js_find_and_click)
if clicked:
    print("✅ Button clicked successfully!")
else:
    print("❌ Button not found!")
    
sleep(2)

# JavaScript to recursively find the <select> element
js_find_select = """
    function findSelect(node) {
        if (!node) return null;
        if (node.tagName === "SELECT" && node.classList.contains("sf-cf-grouped-params")) {
            return node;
        }
        if (node.shadowRoot) {
            const result = findSelect(node.shadowRoot);
            if (result) return result;
        }
        for (const child of node.children || []) {
            const result = findSelect(child);
            if (result) return result;
        }
        return null;
    }
    return findSelect(document.body);
    """

# Get the <select> element from shadow DOM
select_element = driver.execute_script(js_find_select)

sleep(2)

# Click the <select> element
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", select_element)
select_element.click()
sleep(1)

# Send DOWN + ENTER keys
actions = ActionChains(driver)
actions.send_keys(Keys.ARROW_UP).send_keys(Keys.RETURN).perform()

# Get number of options in the dropdown
option_count = driver.execute_script("return arguments[0].options.length;", select_element)

data_rows = []

for i in range(option_count):
    print(f"🔁 Selecting option {i + 1}/{option_count}")

    # Scroll into view and click the dropdown
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", select_element)
    select_element.click()
    sleep(1)

    # Navigate to the desired option with ARROW keys and select it
    actions = ActionChains(driver)
    actions.send_keys(Keys.HOME)  # Move to the first option
    for _ in range(i):
        actions.send_keys(Keys.DOWN)
    actions.send_keys(Keys.RETURN)
    actions.perform()

    sleep(3)  # Wait for UI to update

    # Extract updated price and other sf-cf-number values
    js_extract_label_and_value = """
        function findNumbersWithLabels(node) {
            let results = [];
            if (node.tagName === "SF-CF-NUMBER") {
                let span = node.querySelector("span.sf-cf-number");
                if (span) {
                    results.push({
                        parameterId: node.getAttribute("data-parameterid"),
                        value: span.textContent.trim()
                    });
                }
            }
            if (node.shadowRoot) {
                results = results.concat(findNumbersWithLabels(node.shadowRoot));
            }
            for (const child of node.children || []) {
                results = results.concat(findNumbersWithLabels(child));
            }
            return results;
        }
        return findNumbersWithLabels(document.body);
        """

    results = driver.execute_script(js_extract_label_and_value)
    
    desired_ids = {
        "Price_Leasing",
        "Term",
        "DepositAmountLeasing",
        "MonthlyPaymentWithoutServices",
        "Mileage",
        "TotalPaymentWithoutServices"
    }
    
    # Split into lines and strip empty lines
    lines = [line.strip() for line in all_text.strip().splitlines() if line.strip()]

    # Get the 4th and 5th lines (index 3 and 4)
    line_4 = lines[3]
    line_5 = lines[4]
    
    # data = pd.DataFrame(columns=["Model", "Price_Leasing", "Term", "DepositAmountLeasing", "MonthlyPaymentWithoutServices", "Mileage", "TotalPaymentWithoutServices"])
    
    print(f"Model: {line_4} - Option {i+1}")
    print(len(results))
    row = {"Model": f"{line_4} - Option {i+1}"}
    # Filter and print only the desired values
    
    row_data = {"Model": f"{line_4} - Option {i+1}"} 
    
    for item in results:
        param_id = item['parameterId']
        value = item['value']
        if item["parameterId"] in desired_ids:
            # row[item["parameterId"]] = item["value"]
            row_data[param_id] = value
            print(f"  • {item['parameterId']},: {item['value']}")
    
    data_rows.append(row_data)
    # df = pd.DataFrame(data_rows)

driver.quit()

✅ 'Alle akzeptieren' button clicked.
✅ Button clicked successfully!
🔁 Selecting option 1/7
Model: i5 eDrive40 Limousine - Option 1
58
  • Price_Leasing,: 72.020,00 €
  • Term,: 24 Monate
  • DepositAmountLeasing,: 17.833,51 €
  • MonthlyPaymentWithoutServices,: 579,92 €
  • Mileage,: 5.000 km
  • TotalPaymentWithoutServices,: 31.751,59 €
🔁 Selecting option 2/7
Model: i5 eDrive40 Limousine - Option 2
58
  • Price_Leasing,: 72.020,00 €
  • Term,: 27 Monate
  • DepositAmountLeasing,: 17.833,51 €
  • MonthlyPaymentWithoutServices,: 582,56 €
  • Mileage,: 5.000 km
  • TotalPaymentWithoutServices,: 33.562,63 €
🔁 Selecting option 3/7
Model: i5 eDrive40 Limousine - Option 3
58
  • Price_Leasing,: 72.020,00 €
  • Term,: 30 Monate
  • DepositAmountLeasing,: 17.833,51 €
  • MonthlyPaymentWithoutServices,: 589,70 €
  • Mileage,: 5.000 km
  • TotalPaymentWithoutServices,: 35.524,51 €
🔁 Selecting option 4/7
Model: i5 eDrive40 Limousine - Option 4
58
  • Price_Leasing,: 72.020,00 €
  • Term,: 33 Mona

In [20]:
df = pd.DataFrame(data_rows)

df.to_csv(path+'output/bmw/scraping_v1.csv',index=False)

df

,Model,Price_Leasing,Term,DepositAmountLeasing,MonthlyPaymentWithoutServices,Mileage,TotalPaymentWithoutServices
0,i5 eDrive40 Limousine - Option 1,"72.020,00 €",24 Monate,"17.833,51 €","579,92 €",5.000 km,"31.751,59 €"
1,i5 eDrive40 Limousine - Option 2,"72.020,00 €",27 Monate,"17.833,51 €","582,56 €",5.000 km,"33.562,63 €"
2,i5 eDrive40 Limousine - Option 3,"72.020,00 €",30 Monate,"17.833,51 €","589,70 €",5.000 km,"35.524,51 €"
3,i5 eDrive40 Limousine - Option 4,"72.020,00 €",33 Monate,"17.833,51 €","595,02 €",5.000 km,"37.469,17 €"
4,i5 eDrive40 Limousine - Option 5,"72.020,00 €",36 Monate,"17.833,51 €","599,00 €",5.000 km,"39.397,51 €"
5,i5 eDrive40 Limousine - Option 6,"72.020,00 €",42 Monate,"17.833,51 €","572,73 €",5.000 km,"41.888,17 €"
6,i5 eDrive40 Limousine - Option 7,"72.020,00 €",48 Monate,"17.833,51 €","572,29 €",5.000 km,"45.303,43 €"


# Full Flow 2

In [26]:
url_list_manual = [
    "https://configure.bmw.de/de_DE/configure/I20",
    "https://configure.bmw.de/de_DE/configure/I20/61CF",
    "https://configure.bmw.de/de_DE/configure/U10/51GM",
    "https://configure.bmw.de/de_DE/configure/U11/61EF",
    "https://configure.bmw.de/de_DE/configure/G70/41EJ",
    "https://configure.bmw.de/de_DE/configure/G70/81EH",
    "https://configure.bmw.de/de_DE/configure/G60/61HG",
    "https://configure.bmw.de/de_DE/configure/G60/41HG",
    "https://configure.bmw.de/de_DE/configure/G61/71HH",
    "https://configure.bmw.de/de_DE/configure/G61/61HH",
    "https://configure.bmw.de/de_DE/configure/G26/41HD",
    "https://configure.bmw.de/de_DE/configure/G26/31HD"
]

In [27]:
driver = get_driver(service)

driver.get("https://www.bmw.de/de/konfigurator.html")
time.sleep(2)  # Wait for everything to load

js_find_and_click_cookie = """
    function findAndClickAccept(node) {
        if (!node) return false;

        // Check if it's a button or span containing the "Alle akzeptieren" text
        if ((node.tagName === 'BUTTON' || node.tagName === 'SPAN') && node.innerText.includes("Alle akzeptieren")) {
            node.click();
            return true;
        }

        // Traverse shadow DOM
        if (node.shadowRoot) {
            const found = findAndClickAccept(node.shadowRoot);
            if (found) return true;
        }

        // Traverse regular child elements
        for (const child of node.children || []) {
            const found = findAndClickAccept(child);
            if (found) return true;
        }

        return false;
    }
    return findAndClickAccept(document.body);
"""

# Execute the JavaScript in Selenium
clicked = driver.execute_script(js_find_and_click_cookie)

if clicked:
    print("✅ 'Alle akzeptieren' button clicked.")
else:
    print("❌ Could not find 'Alle akzeptieren' button.")

sleep(2)
    
# JavaScript to extract all <a class="cmp-allmodelscard__link"> inside shadow roots
js_script = """
    function deepQuerySelectorAll(selector) {
        const results = [];

        function traverse(node) {
            if (node.shadowRoot) {
                traverse(node.shadowRoot);
            }

            const elements = node.querySelectorAll(selector);
            if (elements.length) {
                elements.forEach(el => results.push(el));
            }

            // Recurse into child nodes
            node.children && [...node.children].forEach(traverse);
        }

        traverse(document);
        return results;
    }

    return deepQuerySelectorAll("a.cmp-allmodelscard__link").map(a => a.href);
"""


# Run the JS to get all links
car_links = driver.execute_script(js_script)

# Print the result
print(f"Found {len(car_links)} car config URLs:")
url_set = set()
for link in car_links:
    if link not in url_set:
        url_set.add(link)
        
# Close the driver
driver.quit()

✅ 'Alle akzeptieren' button clicked.
Found 5304 car config URLs:


In [28]:
url_list = list(url_set) 
url_list_bmw = [url for url in url_list if url not in url_list_manual]

len(url_list), len(url_list_bmw)

(104, 92)

In [29]:
for url in url_list_bmw[1:]:
    
    driver = get_driver(service)

    # url = "https://configure.bmw.de/de_DE/configure/I20"
    # url = "https://configure.bmw.de/de_DE/configure/U10/51GM/FBLAT,P0C68,S01EE,S0230,S02PA,S02VB,S02VD,S0302,S0428,S0478,S0494,S04GN,S04T2,S04U9,S04V1,S0548,S05AV,S05DM,S0654,S06AE,S06AF,S06PA,S0760,S0801,S0851,S0879,S08R3,S08R9,S08TF/SE000001"
    driver.get(url)

    sleep(2)

    js_find_and_click_cookie = """
        function findAndClickAccept(node) {
            if (!node) return false;

            // Check if it's a button or span containing the "Alle akzeptieren" text
            if ((node.tagName === 'BUTTON' || node.tagName === 'SPAN') && node.innerText.includes("Alle akzeptieren")) {
                node.click();
                return true;
            }

            // Traverse shadow DOM
            if (node.shadowRoot) {
                const found = findAndClickAccept(node.shadowRoot);
                if (found) return true;
            }

            // Traverse regular child elements
            for (const child of node.children || []) {
                const found = findAndClickAccept(child);
                if (found) return true;
            }

            return false;
        }
        return findAndClickAccept(document.body);
    """

    # Execute the JavaScript in Selenium
    clicked = driver.execute_script(js_find_and_click_cookie)

    if clicked:
        print("✅ 'Alle akzeptieren' button clicked.")
    else:
        print("❌ Could not find 'Alle akzeptieren' button.")

    sleep(2)

    body = driver.find_element(By.TAG_NAME, "body")
    all_text = body.text

    # sleep(2)

    js_find_and_click = """
        function findAndClickButton(node) {
            if (!node) return null;

            if (node.tagName === "BUTTON" && node.innerText.includes("Finanzieren & Leasen")) {
                node.click();
                return true;
            }

            if (node.shadowRoot) {
                let found = findAndClickButton(node.shadowRoot);
                if (found) return true;
            }

            for (const child of node.children || []) {
                let found = findAndClickButton(child);
                if (found) return true;
            }

            return false;
        }
        return findAndClickButton(document.body);
        """

    clicked = driver.execute_script(js_find_and_click)
    if clicked:
        print("✅ Button clicked successfully!")
    else:
        print("❌ Button not found!")
        
    sleep(2)

    # JavaScript to find <select> elements with specific aria-labels
    js_find_target_selects = """
        function findMatchingSelects(node, labels = [], result = []) {
            if (!node) return result;
            if (node.tagName === "SELECT") {
                const label = node.getAttribute("aria-label");
                if (label === "Laufzeit" || label === "Laufleistung p.a.") {
                    result.push(node);
                }
            }
            if (node.shadowRoot) findMatchingSelects(node.shadowRoot, labels, result);
            for (const child of node.children || []) {
                findMatchingSelects(child, labels, result);
            }
            return result;
        }
        return findMatchingSelects(document.body);
    """
    # Get the matching <select> elements (Laufzeit and Laufleistung p.a.)
    select_elements = driver.execute_script(js_find_target_selects)

    # Get options for each select
    all_options = []
    for sel in select_elements:
        values = driver.execute_script("return Array.from(arguments[0].options).map(o => o.value);", sel)
        all_options.append(values)

    data_rows = []

    # Loop through all combinations
    for combo in product(*all_options):
        print(f"\n🔁 Trying combo: {combo}")

        # Re-fetch the two target selects again to avoid stale elements
        select_elements = driver.execute_script(js_find_target_selects)

        for i, sel in enumerate(select_elements):
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", sel)
            sel.click()
            sleep(1.5)

            # Find index of desired value
            index = driver.execute_script("""
                const options = Array.from(arguments[0].options);
                return options.findIndex(opt => opt.value === arguments[1]);
            """, sel, combo[i])

            if index == -1:
                continue

            # Use ARROW keys to move to correct option
            actions = ActionChains(driver)
            actions.send_keys(Keys.HOME)
            for _ in range(index):
                actions.send_keys(Keys.DOWN)
            actions.send_keys(Keys.RETURN).perform()
            sleep(2)

        # Extract result values
        js_extract_label_and_value = """
        function findNumbersWithLabels(node) {
            let results = [];
            if (node.tagName === "SF-CF-NUMBER") {
                let span = node.querySelector("span.sf-cf-number");
                if (span) {
                    results.push({
                        parameterId: node.getAttribute("data-parameterid"),
                        value: span.textContent.trim()
                    });
                }
            }
            if (node.shadowRoot) {
                results = results.concat(findNumbersWithLabels(node.shadowRoot));
            }
            for (const child of node.children || []) {
                results = results.concat(findNumbersWithLabels(child));
            }
            return results;
        }
        return findNumbersWithLabels(document.body);
        """
        results = driver.execute_script(js_extract_label_and_value)

        desired_ids = {
            "Price_Leasing", "Term", "DepositAmountLeasing",
            "MonthlyPaymentWithoutServices", "Mileage", "TotalPaymentWithoutServices"
        }
        
        # Split into lines and strip empty lines
        lines = [line.strip() for line in all_text.strip().splitlines() if line.strip()]

        # Get the 4th and 5th lines (index 3 and 4)
        model_name = lines[3]
        print(f"Model: {model_name} - Option {combo}")
        
        row_data = {"Model": f"{model_name} - Option {combo}"}    

        for item in results:
            param_id = item['parameterId']
            value = item['value']
            if item["parameterId"] in desired_ids:
                row_data[param_id] = value
                print(f"  • {item['parameterId']}: {item['value']}")
        
        data_rows.append(row_data)
        
        df = pd.DataFrame(data_rows)
        df.to_csv(path+f'output/bmw/scraping_{model_name}.csv',index=False)
        
    driver.quit()

✅ 'Alle akzeptieren' button clicked.
✅ Button clicked successfully!

🔁 Trying combo: ('24', '5000')
Model: BMW X6 M60i xDrive - Option ('24', '5000')
  • Price_Leasing: 124.570,00 €
  • Term: 24 Monate
  • DepositAmountLeasing: 24.914,00 €
  • MonthlyPaymentWithoutServices: 1.335,87 €
  • Mileage: 5.000 km
  • TotalPaymentWithoutServices: 56.974,88 €

🔁 Trying combo: ('24', '10000')
Model: BMW X6 M60i xDrive - Option ('24', '10000')
  • Price_Leasing: 124.570,00 €
  • Term: 24 Monate
  • DepositAmountLeasing: 24.914,00 €
  • MonthlyPaymentWithoutServices: 1.384,87 €
  • Mileage: 10.000 km
  • TotalPaymentWithoutServices: 58.150,88 €

🔁 Trying combo: ('24', '15000')
Model: BMW X6 M60i xDrive - Option ('24', '15000')
  • Price_Leasing: 124.570,00 €
  • Term: 24 Monate
  • DepositAmountLeasing: 24.914,00 €
  • MonthlyPaymentWithoutServices: 1.433,87 €
  • Mileage: 15.000 km
  • TotalPaymentWithoutServices: 59.326,88 €

🔁 Trying combo: ('24', '20000')
Model: BMW X6 M60i xDrive - Option ('2

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=136.0.7103.93)
Stacktrace:
	GetHandleVerifier [0x002AFC53+61635]
	GetHandleVerifier [0x002AFC94+61700]
	(No symbol) [0x000D05D3]
	(No symbol) [0x000BFE20]
	(No symbol) [0x000DDD1F]
	(No symbol) [0x00143E8C]
	(No symbol) [0x0015DF19]
	(No symbol) [0x0013D096]
	(No symbol) [0x0010C840]
	(No symbol) [0x0010D6A4]
	GetHandleVerifier [0x00534573+2701795]
	GetHandleVerifier [0x0052FCF6+2683238]
	GetHandleVerifier [0x0054AA3E+2793134]
	GetHandleVerifier [0x002C6915+155013]
	GetHandleVerifier [0x002CCFFD+181357]
	GetHandleVerifier [0x002B74A8+92440]
	GetHandleVerifier [0x002B7650+92864]
	GetHandleVerifier [0x002A2040+5296]
	BaseThreadInitThunk [0x75FA7BA9+25]
	RtlInitializeExceptionChain [0x77A3C36B+107]
	RtlClearBits [0x77A3C2EF+191]
	(No symbol) [0x00000000]


In [60]:
df = pd.DataFrame(data_rows)

df.to_csv(path+f'output/bmw/scraping_{model_name}.csv',index=False)

df

,Model,Price_Leasing,Term,DepositAmountLeasing,MonthlyPaymentWithoutServices,Mileage,TotalPaymentWithoutServices
0,"i5 eDrive40 Limousine - Option ('24', '5000')","72.020,00 €",24 Monate,"17.833,51 €","579,92 €",5.000 km,"31.751,59 €"
1,"i5 eDrive40 Limousine - Option ('24', '10000')","72.020,00 €",24 Monate,"17.833,51 €","636,78 €",10.000 km,"33.116,23 €"
2,"i5 eDrive40 Limousine - Option ('24', '15000')","72.020,00 €",24 Monate,"17.833,51 €","665,20 €",15.000 km,"33.798,31 €"
3,"i5 eDrive40 Limousine - Option ('24', '20000')","72.020,00 €",24 Monate,"17.833,51 €","693,63 €",20.000 km,"34.480,63 €"
4,"i5 eDrive40 Limousine - Option ('24', '25000')","72.020,00 €",24 Monate,"17.833,51 €","722,06 €",25.000 km,"35.162,95 €"
...,...,...,...,...,...,...,...
107,"i5 eDrive40 Limousine - Option ('48', '60000')","72.020,00 €",48 Monate,"17.833,51 €","909,37 €",50.000 km,"61.483,27 €"
108,"i5 eDrive40 Limousine - Option ('48', '65000')","72.020,00 €",48 Monate,"17.833,51 €","909,37 €",50.000 km,"61.483,27 €"
109,"i5 eDrive40 Limousine - Option ('48', '70000')","72.020,00 €",48 Monate,"17.833,51 €","909,37 €",50.000 km,"61.483,27 €"
110,"i5 eDrive40 Limousine - Option ('48', '75000')","72.020,00 €",48 Monate,"17.833,51 €","909,37 €",50.000 km,"61.483,27 €"


# Test full combo 

In [21]:
url = "https://configure.bmw.de/de_DE/configure/I20"


driver = get_driver(service)
driver.get(url)
sleep(2)

# Accept Cookies
js_click_cookie = """
    function findAndClickAccept(node) {
        if (!node) return false;
        if ((node.tagName === 'BUTTON' || node.tagName === 'SPAN') && node.innerText.includes("Alle akzeptieren")) {
            node.click();
            return true;
        }
        if (node.shadowRoot) {
            return findAndClickAccept(node.shadowRoot);
        }
        for (const child of node.children || []) {
            if (findAndClickAccept(child)) return true;
        }
        return false;
    }
    return findAndClickAccept(document.body);
"""

clicked = driver.execute_script(js_click_cookie)
print("✅ Cookie accepted" if clicked else "❌ Cookie not found")
sleep(2)

✅ Cookie accepted


In [22]:
# Click "Finanzieren & Leasen"
js_find_and_click = """
    function findAndClickButton(node) {
        if (!node) return null;

        if (node.tagName === "BUTTON" && node.innerText.includes("Finanzieren & Leasen")) {
            node.click();
            return true;
        }

        if (node.shadowRoot) {
            let found = findAndClickButton(node.shadowRoot);
            if (found) return true;
        }

        for (const child of node.children || []) {
            let found = findAndClickButton(child);
            if (found) return true;
        }

        return false;
    }
    return findAndClickButton(document.body);
    """

clicked = driver.execute_script(js_find_and_click)
if clicked:
    print("✅ Button clicked successfully!")
else:
    print("❌ Button not found!")
    
sleep(2)

✅ Button clicked successfully!


In [24]:
select_elements = driver.execute_script(js_find_all_selects)

# Extract options for each select
all_options = []
for sel in select_elements:
    values = driver.execute_script("return Array.from(arguments[0].options).map(o => o.value);", sel)
    all_options.append(values)

print("Dropdowns found:", len(all_options))
for i, opts in enumerate(all_options):
    print(f"Dropdown {i+1}: {opts}")

Dropdowns found: 4
Dropdown 1: ['1009519', '1008961']
Dropdown 2: ['24', '27', '30', '33', '36', '42', '48']
Dropdown 3: ['5000', '10000', '15000', '20000', '25000', '30000', '35000', '40000', '45000', '50000', '55000', '60000', '65000', '70000', '75000', '80000']
Dropdown 4: ['Leasingratenversicherung Tod und Arbeitsunfähigkeit *', 'Leasingratenversicherung Tod und Arbeitsunfähigkeit, Arbeitslosigkeit/Schwere Krankheit **']


In [25]:
data_rows = []

# Try all combinations
for combo in product(*all_options):
    print(f"\n🔁 Trying combo: {combo}")
    select_elements = driver.execute_script(js_find_all_selects)

    for i, sel in enumerate(select_elements):
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", sel)
        sel.click()
        sleep(1.5)

        index = driver.execute_script("""
            const options = Array.from(arguments[0].options);
            return options.findIndex(opt => opt.value === arguments[1]);
        """, sel, combo[i])

        if index == -1:
            print(f"⚠️ Value {combo[i]} not found in dropdown {i}")
            continue

        actions = ActionChains(driver)
        actions.send_keys(Keys.HOME)
        for _ in range(index):
            actions.send_keys(Keys.DOWN)
        actions.send_keys(Keys.RETURN).perform()
        sleep(2)

    # Extract price data
    js_extract_label_and_value = """
        function findNumbersWithLabels(node) {
            let results = [];
            if (node.tagName === "SF-CF-NUMBER") {
                let span = node.querySelector("span.sf-cf-number");
                if (span) {
                    results.push({
                        parameterId: node.getAttribute("data-parameterid"),
                        value: span.textContent.trim()
                    });
                }
            }
            if (node.shadowRoot) {
                results = results.concat(findNumbersWithLabels(node.shadowRoot));
            }
            for (const child of node.children || []) {
                results = results.concat(findNumbersWithLabels(child));
            }
            return results;
        }
        return findNumbersWithLabels(document.body);
    """
    results = driver.execute_script(js_extract_label_and_value)

    desired_ids = {
        "Price_Leasing", "Term", "DepositAmountLeasing",
        "MonthlyPaymentWithoutServices", "Mileage", "TotalPaymentWithoutServices"
    }

    body = driver.find_element(By.TAG_NAME, "body")
    lines = [line.strip() for line in body.text.strip().splitlines() if line.strip()]
    model_name = lines[3] if len(lines) > 3 else "Unknown Model"
    print(f"Model: {model_name} - Option {combo}")

    row_data = {"Model": f"{model_name} - Option {combo}"}
    for item in results:
        if item["parameterId"] in desired_ids:
            row_data[item["parameterId"]] = item["value"]
            print(f"  • {item['parameterId']}: {item['value']}")

    data_rows.append(row_data)

df = pd.DataFrame(data_rows)
df.to_csv(f"output/bmw/scraping_{model_name}.csv", index=False)
driver.quit()


🔁 Trying combo: ('1009519', '24', '5000', 'Leasingratenversicherung Tod und Arbeitsunfähigkeit *')


StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=136.0.7103.93)
Stacktrace:
	GetHandleVerifier [0x002AFC53+61635]
	GetHandleVerifier [0x002AFC94+61700]
	(No symbol) [0x000D05D3]
	(No symbol) [0x000D6DD0]
	(No symbol) [0x000D9335]
	(No symbol) [0x0015F531]
	(No symbol) [0x0013D29C]
	(No symbol) [0x0015E61B]
	(No symbol) [0x0013D096]
	(No symbol) [0x0010C840]
	(No symbol) [0x0010D6A4]
	GetHandleVerifier [0x00534573+2701795]
	GetHandleVerifier [0x0052FCF6+2683238]
	GetHandleVerifier [0x0054AA3E+2793134]
	GetHandleVerifier [0x002C6915+155013]
	GetHandleVerifier [0x002CCFFD+181357]
	GetHandleVerifier [0x002B74A8+92440]
	GetHandleVerifier [0x002B7650+92864]
	GetHandleVerifier [0x002A2040+5296]
	BaseThreadInitThunk [0x75FA7BA9+25]
	RtlInitializeExceptionChain [0x77A3C36B+107]
	RtlClearBits [0x77A3C2EF+191]
	(No symbol) [0x00000000]
